In [1]:
import pandas as pd
import numpy as np

In [35]:
data = pd.read_pickle("../data/products_data.pickle")
cols = data[0]
rows = data[1:]
df = pd.DataFrame(rows, columns = cols)
df.head()

,product_id,name,caption,cat_1,cat_2,cat_3,url,capacity,current_price,regular_price,lowest_price_last_30_days,price_per_unit,unit,badges,ingredients,rating,reviews
0,41060,GOT2B Glued,"Klej do włosów, poziom utrwalenia 6, wodoodporny",Włosy,Stylizacja włosów,Żele do włosów,https://www.rossmann.pl/Produkt/Zele-do-wlosow...,150 ml,"27,99 zł",,,"18,66 zł",100 ml,,"Ingredients: Aqua (Water, Eau), Alcohol denat....",4.8,35
1,41060,GOT2B Glued,"Klej do włosów, poziom utrwalenia 6, wodoodporny",Włosy,Stylizacja włosów,Żele do włosów,https://www.rossmann.pl/Produkt/Zele-do-wlosow...,150 ml,"27,99 zł",,,"18,66 zł",100 ml,,"Ingredients: Aqua (Water, Eau), Alcohol denat....",4.8,35
2,133259,ZIAJA Bloker,Antyperspirant w kulce,Mężczyzna,Higiena,Dezodoranty i antyperspiranty,https://www.rossmann.pl/Produkt/Antyperspirant...,60 ml,"6,99 zł",,,"11,65 zł",100 ml,,"Aqua (Water), Aluminum Chloride, Glycerin, PPG...",4.9,77
3,133259,ZIAJA Bloker,Antyperspirant w kulce,Mężczyzna,Higiena,Dezodoranty i antyperspiranty,https://www.rossmann.pl/Produkt/Antyperspirant...,60 ml,"6,99 zł",,,"11,65 zł",100 ml,,"Aqua (Water), Aluminum Chloride, Glycerin, PPG...",4.9,77
4,2085510,PHILIPS OneBlade 360,Maszynka do golenia QP2724/23,Pielęgnacja i higiena,Ciało,Golenie i depilacja ciała,https://www.rossmann.pl/Produkt/Golarki-i-masz...,1 szt.,"169,99 zł",,,"169,99 zł",1 szt.,,,4.9,18
